Reference:
1. https://github.com/kurchi1205/Next-word-Prediction-using-Swiftkey-Data/blob/main/GPT%20Model.ipynb

2. https://towardsdatascience.com/fine-tune-a-non-english-gpt-2-model-with-huggingface-9acc2dc7635b

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.0 MB/s eta 0:00:00


In [ ]:
#importing libraries to be used
import nltk
import re
from tqdm import tqdm
import numpy as np
from nltk import tokenize
import pickle
from nltk.tokenize import word_tokenize

# Code Reference : https://towardsdatascience.com/fine-tune-a-non-english-gpt-2-model-with-huggingface-9acc2dc7635b
from transformers import OpenAIGPTTokenizer,OpenAIGPTLMHeadModel,TextDataset,TrainingArguments,Trainer,pipeline,DataCollatorForLanguageModeling

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
path = '/content/drive/MyDrive/EE769/Project/data.txt' #path of dataset
data_set = open(path).read().lower() #opening the file and saving as array
text_corpus = tokenize.sent_tokenize(data_set) #tokenizing the dataset
print(len(text_corpus))

4807


In [ ]:
#using regex to clean the data
def white_space(t):  #removes extra white spaces
  return re.sub("\s+", " ", t)
def special_character(t): #removes special character
  return re.sub("[^0-9A-Za-z]", "", t)

for i in range(len(text_corpus)):
  text_corpus[i] = white_space(text_corpus[i])
  text_corpus[i] = special_character(text_corpus[i])

count=[]
for i in tqdm(text_corpus):
    count.append(len(i.split()))
print(np.mean(count))

100%|██████████| 4807/4807 [00:00<00:00, 998960.48it/s]

1.0


In [ ]:
train_set = text_corpus[:int(0.9*len(text_corpus))] #training dataset
test_set = text_corpus[int(0.9*len(text_corpus)):] #testing dataset

train_set = '.'.join(train_set)
test_set = '.'.join(test_set)

with open("train_set.txt", "w") as fp: #pickling
  fp.write(train_set)
with open("test_set.txt", "w") as fp: #pickling
  fp.write(train_set)

In [ ]:
tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt") #using OpenAIGPTTokenizer

print('vocabulary size :', tokenizer.vocab_size) #checking vocabulary size of the model
print('Max Sequence length :', tokenizer.model_max_length) #checking max length of the sentence

data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False) #setting for language modeling, during training it will generate train and test data

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


vocabulary size : 40478
Max Sequence length : 512


In [ ]:
train_data_set = TextDataset(tokenizer = tokenizer, file_path = 'train_set.txt', overwrite_cache = True, block_size = 19) #convetring tokens to ids
test_data_set = TextDataset(tokenizer = tokenizer, file_path = 'test_set.txt', overwrite_cache = True, block_size = 19)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt') #using pretrained GPT model

In [ ]:
training_args = TrainingArguments( output_dir = 'gpt_model', overwrite_output_dir = True, per_device_train_batch_size = 64, per_device_eval_batch_size = 64, learning_rate = 0.0005, num_train_epochs = 3) #setting arguments for training
trainer = Trainer(model = model, args = training_args, data_collator = data_collator, train_dataset = train_data_set, eval_dataset = test_data_set) #Initiallizing trainer class object which will do the training

trainer.train() #training the model for 5 epochs

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=6, training_loss=6.438040415445964, metrics={'train_runtime': 65.1246, 'train_samples_per_second': 3.593, 'train_steps_per_second': 0.092, 'total_flos': 2268912881664.0, 'train_loss': 6.438040415445964, 'epoch': 3.0})

In [ ]:
trainer.save_model() #saving the model

In [ ]:
trainer.evaluate(test_data_set) #evaluating the model using the test data set

{'eval_loss': 3.2083466053009033,
 'eval_runtime': 7.9289,
 'eval_samples_per_second': 9.837,
 'eval_steps_per_second': 0.252,
 'epoch': 3.0}

In [ ]:
generator = pipeline('text-generation', tokenizer = 'openai-gpt', model = 'gpt_model')

In [ ]:
def predict_next():
    tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")
    model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt')
    generator = pipeline('text-generation', tokenizer='openai-gpt', model='gpt_model') 

    text = input('Enter the text: ')
    length= len(tokenizer.encode(text, return_tensors='pt')[0])
        
    max_length = length+1
    
    print('Next Word: ')
    print(generator(text , max_length=max_length)[0]['generated_text'].split(' ')[-1])
    print(generator(text , max_length=max_length , num_beams = 5)[0]['generated_text'].split(' ')[-1])
    print(generator(text , max_length=max_length , do_sample=True,temperature = 0.7)[0]['generated_text'].split(' ')[-1])

In [ ]:
predict_next()

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


Enter the text: What are you
Next Word: 
doin
doing
doin'


In [ ]:
predict_next()

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


Enter the text: The dog is
Next Word: 
fol
still
asle
